Goal: separate the ingrdients in the FoodKG table into a list of ingredients

In [1]:
# !pip install --upgrade pip -q
# !pip install psycopg2-binary -q
# !pip install SQLAlchemy -q
# !pip install spacy -q
# !python -m spacy download en_core_web_sm -q

In [2]:
import json
import os

import pandas as pd
from dotenv import load_dotenv

from util import process_branded_food_product_df, assemble_branded_food_product_df

# from sqlalchemy import create_engine

load_dotenv()  # Take environment variables from .env
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",  # local model
    api_key=api_key
)

In [3]:
# # Credentials
# host = os.getenv("POSTGRES_HOST")
# port = os.getenv("POSTGRES_PORT")
# database = os.getenv("POSTGRES_DATABASE")
# user = os.getenv("POSTGRES_USER")
# password = os.getenv("POSTGRES_PASSWORD")
# # Create the connection engine
# engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")
# # Test the connection
# try:
#     with engine.connect() as conn:
#         print("✅ Connected successfully!")
# except Exception as e:
#     print("❌ Connection failed:", e)

In [4]:
# # Test: query 5 rows
# query = "SELECT * FROM \"FoodKG\" LIMIT 5;"  # quotes protect uppercase table names
# df = pd.read_sql(query, engine)
# # display nicely
# pd.set_option('display.max_columns', None)  # show all columns
# pd.set_option('display.max_rows', 5)  # show up to 5 rows
# display(df)

In [5]:
df = pd.read_csv("data/usda_2022_branded_food_product_INGREDIENTS_ONLY.csv")

In [6]:
df.head()

,fdc_id,ingredients
0,344604,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,..."
1,344605,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,..."
2,344606,"White Turkey, Natural Flavoring"
3,344607,"Turkey Breast, Natural Flavoring"
4,344608,"Turkey, natural Flavoring."


In [7]:
process_branded_food_product_df(
    df=df,
    model="qwen/qwen3-4b-2507",
    client=client,
    batch_size=10,
)

Wrote part_344644_344653.csv (0:10) — checkpoint=344653
Wrote part_344654_344663.csv (10:20) — checkpoint=344663
Wrote part_344664_344673.csv (20:30) — checkpoint=344673
Done.


In [8]:
final_df = assemble_branded_food_product_df()

In [9]:
len(final_df)

70

In [10]:
# CSV can’t store native Python lists (they are objects), so use JSON dump
product_table = pd.DataFrame({
    "fdc_id": final_df["fdc_id"],
    "ingredients_normalized": final_df["ingredients_normalized"].apply(json.dumps),
}).sort_values("fdc_id")
# Create directory if it doesn't exist and save
os.makedirs("sample", exist_ok=True)
product_table.to_csv("sample/branded_food_product_ingredients_normalized.csv", index=False)

In [11]:
product_table.tail()

,fdc_id,ingredients_normalized
65,344669,"""['flour', 'soybean oil', 'palm oil', 'cheese'..."
66,344670,"""['flour', 'soybean oil', 'palm oil', 'cheese'..."
67,344671,"""['flour', 'oil', 'cheese', 'salt', 'paprika',..."
68,344672,"""['flour', 'soybean oil', 'palm oil', 'cheese'..."
69,344673,"""['flour', 'soybean oil', 'palm oil', 'cheese'..."
